In [36]:
import warnings
warnings.filterwarnings("ignore")

from tensorflow.keras.models import load_model
import tensorflow as tf
import os
import cv2
import shutil
import random
import numpy as np
from tensorflow.keras.utils import normalize
from tqdm import tqdm
import tensorflow
from skimage.io import imread, imshow
from tensorflow.keras import backend as K
from skimage.transform import resize
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import MeanIoU

seed = 42
random.seed = seed
np.random.seed = seed
tf.random.set_seed(seed)

In [3]:
model = load_model("model/UNET_model.h5", compile=False)

## Mask Generation

In [12]:
IMG_WIDTH = 224
IMG_HEIGHT = 224
IMG_CHANNELS = 3

In [13]:
path = "phase_1_data/original/"

In [14]:
image_names = os.listdir(path)
image_names.sort()

In [15]:
print(image_names)

['abnormal_Pat1_0.png', 'abnormal_Pat1_1.png', 'abnormal_Pat1_2.png', 'abnormal_Pat1_3.png', 'abnormal_Pat1_4.png', 'abnormal_Pat1_5.png', 'abnormal_Pat1_6.png', 'abnormal_Pat1_7.png', 'abnormal_Pat1_8.png', 'abnormal_Pat1_9.png', 'abnormal_pat 10_0.png', 'abnormal_pat 10_1.png', 'abnormal_pat 10_10.png', 'abnormal_pat 10_11.png', 'abnormal_pat 10_12.png', 'abnormal_pat 10_13.png', 'abnormal_pat 10_14.png', 'abnormal_pat 10_15.png', 'abnormal_pat 10_16.png', 'abnormal_pat 10_2.png', 'abnormal_pat 10_3.png', 'abnormal_pat 10_4.png', 'abnormal_pat 10_5.png', 'abnormal_pat 10_6.png', 'abnormal_pat 10_7.png', 'abnormal_pat 10_8.png', 'abnormal_pat 10_9.png', 'abnormal_pat 11_0.png', 'abnormal_pat 11_1.png', 'abnormal_pat 11_10.png', 'abnormal_pat 11_11.png', 'abnormal_pat 11_12.png', 'abnormal_pat 11_13.png', 'abnormal_pat 11_14.png', 'abnormal_pat 11_15.png', 'abnormal_pat 11_2.png', 'abnormal_pat 11_3.png', 'abnormal_pat 11_4.png', 'abnormal_pat 11_5.png', 'abnormal_pat 11_6.png', 'abnor

In [37]:
outdir = "phase_2_data"

if outdir in os.listdir("."):
    shutil.rmtree(outdir)
    os.makedirs(os.path.join(outdir, 'Normal'))
    os.makedirs(os.path.join(outdir, 'Abnormal'))
else:
    os.makedirs(os.path.join(outdir, 'Normal'))
    os.makedirs(os.path.join(outdir, 'Abnormal'))

In [16]:
original_images = np.zeros((len(image_names), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

for n, id_ in tqdm(enumerate(image_names), total=len(image_names)):
    image_path = path + id_
    img = imread(image_path)[:,:,:3]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    original_images[n] = img

100%|██████████████████████████████████████████████████████████████████████████████| 1428/1428 [01:27<00:00, 16.37it/s]


In [17]:
prediction = model.predict(original_images, batch_size=32, verbose=1)

45/45 [==============================] - 133s 3s/step


In [33]:
pred = (prediction > 0.3)

In [38]:
for i in range(len(prediction)):
    image = cv2.imread(os.path.join('phase_1_data/original', image_names[i]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_mask = (pred[i]*255).astype(np.uint8)
    
    ret, mask = cv2.threshold(image_mask, 127, 255, cv2.THRESH_BINARY)
    extracted = cv2.bitwise_or(image, image, mask=image_mask)
    if image_names[i].split("_")[0] == 'abnormal':
        save_path = os.path.join(outdir, 'Abnormal', image_names[i])
        cv2.imwrite(save_path, extracted)
    elif image_names[i].split("_")[0] == 'normal':
        save_path = os.path.join(outdir, 'Normal', image_names[i])
        cv2.imwrite(save_path, extracted)
    else:
        print("Check the filename")

## Thank You